In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tqdm

In [20]:
#nhập dữ liệu
df = pd.read_csv("data.csv", delimiter = ";")
pd.set_option('display.width', 500)
print(df.head())

   Marital status  Application mode  Application order  Course  Daytime/evening attendance\t  Previous qualification  Previous qualification (grade)  Nacionality  Mother's qualification  Father's qualification  ...  Curricular units 2nd sem (credited)  Curricular units 2nd sem (enrolled)  Curricular units 2nd sem (evaluations)  Curricular units 2nd sem (approved)  Curricular units 2nd sem (grade)  Curricular units 2nd sem (without evaluations)  Unemployment rate  Inflation rate   GDP    Target
0               1                17                  5     171                             1                       1                           122.0            1                      19                      12  ...                                    0                                    0                                       0                                    0                          0.000000                                               0               10.8             1.4  1.74   Dropout
1 

In [21]:
#chuẩn hóa dữ liệu
df.columns = df.columns.str.strip()
#chuyển target thành nhị phân 0, 1 
df["Target"] = df["Target"].map({"Graduate" : 1, "Enrolled" : 0, "Dropout" : 0})

#Xác định X và y
X = df.drop(columns = ["Target"])
y = df["Target"]

#Chia dataset thành 3 phần theo tỉ lệ 8:1:1
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 42)

In [26]:
#class Logistic Regression
class LogisticRegression:
    def __init__(self, epochs: int, lr: float) -> None:
        self.lr = lr
        self.epochs = epochs
        self.losses = []
        self.metric = []
    #tạo hàm sigmoid để biến giá trị đầu ra thành 0 đến 1
    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        return 1/(1 + np.exp(-z))
    #tạo hàm mất mát để đo độ chênh lệch giữa dự đoán vs. thực tế
    def loss_fn(self, y: np.ndarray, y_hat: np.ndarray) -> float:
        return -(y*np.log(y_hat) + (1 - y)*np.log(1 - y_hat)).mean()
    #tạo hàm accuracy để đánh giá mô hình
    def accuracy(self, y: np.ndarray, y_hat: np.ndarray) -> float:
        return (1 - np.abs((y - y_hat)).mean())

    def predict(self, X: np.ndarray) -> np.ndarray:
        z = np.dot(X, self.theta)
        return self.sigmoid(z)
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        n = X.shape[0]
        d = X.shape[1]
        self.theta = np.zeros((d , 1))
        y = y.values.reshape(-1, 1)
        with tqdm.tqdm(range(self.epochs)) as pb:
            for e in pb:
                #forward
                pb.set_description(f"epochs {e + 1}")
                y_hat = self.predict(X)
                #backward
                diff = y_hat - y
                gradient = 1/n * np.matmul(X.T, diff) #(d, 1)
                self.theta -= self.lr*gradient

                #evaluating
                y_hat = self.predict(X)
                loss = self.loss_fn(y, y_hat)
                acc = self.accuracy(y, y_hat)
                pb.set_postfix({
                    "loss": loss,
                    "acc": acc
                })

                self.losses.append(loss)
                self.metric.append(acc)

In [32]:
#huấn luyện mô hình và kiểm tra trên tập dev
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

model = LogisticRegression(lr=0.01, epochs=1000)
model.fit(X_train, y_train)

y_pred_dev = model.predict(X_dev).flatten()
y_pred_dev_labels = (y_pred_dev >= 0.5).astype(int)

accuracy_dev = model.accuracy(y_dev, y_pred_dev_labels)
print(f"Accuracy trên tập dev: {accuracy_dev:.2f}")

epochs 1000: 100%|██████████| 1000/1000 [00:01<00:00, 598.50it/s, loss=0.384, acc=0.732]

Accuracy trên tập dev: 0.84


In [ ]:
#kiểm tra trên tập test
y_pred_test = model.predict(X_test).flatten()
y_pred_test_labels = (y_pred_test >= 0.5).astype(int)

accuracy_test = model.accuracy(y_test, y_pred_test_labels)
print(f"Accuracy trên tập test: {accuracy_test:.2f}")

Accuracy trên tập test: 0.80%
